In [1]:
import pandas as pd

In [28]:
# Columns: account_id, name, point, call_count, total_call_time, total_exclusive_time, min_call_time, max_call_time, sum_of_squares, instances, language, app_name, app_id, scope, host, display_host, pid, agent_version, labels
# do not exepct index column
data = pd.read_csv('.././data/metrics_collector.tsv', sep='\t',names=['account_id', 'name', 'point', 'call_count', 'total_call_time', 'total_exclusive_time', 'min_call_time', 'max_call_time', 'sum_of_squares', 'instances', 'language', 'app_name', 'app_id', 'scope', 'host', 'display_host', 'pid', 'agent_version', 'labels'], header=None)
data['point'] = pd.to_datetime(data['point'])
data['scope'].fillna('', inplace=True)

C:\Users\germi\AppData\Local\Temp\ipykernel_2476\2427093020.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('.././data/metrics_collector.tsv', sep='\t',names=['account_id', 'name', 'point', 'call_count', 'total_call_time', 'total_exclusive_time', 'min_call_time', 'max_call_time', 'sum_of_squares', 'instances', 'language', 'app_name', 'app_id', 'scope', 'host', 'display_host', 'pid', 'agent_version', 'labels'], header=None)


In [43]:
filtered_data = data[
    (data['language'] == 'java') &
    (data['app_name'] == '[GMonit] Collector') &
    (data['scope'] == '') &
    (data['name'] == 'HttpDispatcher')
]

# Group by 'point' and calculate the sum and mean
web_response = filtered_data.groupby('point').apply(
    lambda x: pd.Series({
        'web_response': x['total_call_time'].sum() / x['call_count'].sum()
    })
).reset_index()
web_response = web_response.sort_values(by='point')


In [47]:
filtered_data = data[
    (data['language'] == 'java') &
    (data['app_name'] == '[GMonit] Collector') &
    (data['scope'] == '') &
    (data['name'] == 'HttpDispatcher')
]
throughput = filtered_data.groupby('point')['call_count'].sum().reset_index()
throughput.columns = ['point', 'sum_call_count']
throughput  = throughput.sort_values(by='point')


In [49]:
import pandas as pd

filtered_data = data[
    (data['language'] == 'java') &
    (data['app_name'] == '[GMonit] Collector') &
    (data['scope'] == '') &
    (data['name'] == 'Apdex')
]
grouped_data = filtered_data.groupby('point').agg({
    'call_count': 'sum',
    'total_call_time': 'sum',
    'total_exclusive_time': 'sum'
}).reset_index()
grouped_data[' '] = (grouped_data['call_count'] + grouped_data['total_call_time'] / 2) / (
    grouped_data['call_count'] + grouped_data['total_call_time'] + grouped_data['total_exclusive_time']
)
grouped_data = grouped_data.rename(columns={'point': 'time'})
apdex = grouped_data[['time', ' ']]
apdex = apdex.sort_values(by='time')


In [50]:
apdex

,time,
0,2024-04-15 23:32:00,0.999679
1,2024-04-15 23:33:00,0.999745
2,2024-04-15 23:34:00,0.999687
3,2024-04-15 23:35:00,0.999812
4,2024-04-15 23:36:00,0.999489
...,...,...
43267,2024-05-16 00:54:00,0.998828
43268,2024-05-16 00:55:00,0.998495
43269,2024-05-16 00:56:00,0.997736
43270,2024-05-16 00:57:00,0.997999
